In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import os
sys.path.append('../../scripts')
from datasets_gen import CustomDataset
from simple_nn import SimpleNN, test, run_model
from federated_functions import average_model_weights

In [13]:
df = pd.read_csv("../../datasets/telemetry_analysis/processed/telemetry_analysis.csv", index_col=[0])
X = df.drop(columns= ['encoded_errors', 'machineID'])
Y = df['encoded_errors']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
train_dataset = CustomDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

input_size = X.shape[1]
hidden_size = 10
num_classes = 2

## Federated Models

In [4]:
fed_model_path = os.listdir("./federated_models/.")

In [7]:
weights = []
for weight in fed_model_path:
    weights.append(torch.load(f"federated_models/{weight}", weights_only=True))

In [8]:
weights

[OrderedDict([('fc1.weight',
               tensor([[-0.1369, -0.4833,  0.2445, -0.0954],
                       [ 0.0524,  0.0314,  0.0333,  0.3736],
                       [-0.2641, -0.3768,  0.1026,  0.2117],
                       [-0.0337,  1.2335, -0.8771, -0.1039],
                       [-0.3538, -0.0901,  0.1150,  0.0294],
                       [ 0.2482, -0.3616,  0.3140, -0.3399],
                       [ 0.0482,  0.8003, -0.2887,  0.1160],
                       [ 0.1926, -0.0124,  0.0478, -0.1305],
                       [ 0.3555,  0.1806, -0.3018,  0.1723],
                       [-0.4603,  1.1894, -0.8348, -0.1111]])),
              ('fc1.bias',
               tensor([-0.2006, -0.2817, -0.4494, -0.0400,  0.0250,  0.0700, -0.3336, -0.2605,
                       -0.4029,  0.2576])),
              ('fc2.weight',
               tensor([[-0.0380, -0.1150, -0.1027, -0.2482, -0.2108, -0.0345, -0.1736, -0.2775,
                         0.0833, -0.4276],
                       [

In [9]:
average_weights = average_model_weights(weight_list=weights)
new_model = SimpleNN(input_size, hidden_size, num_classes)
new_model.load_state_dict(average_weights)

<All keys matched successfully>

In [14]:
test(new_model, testloader=train_loader)

c:\Users\ragha\anaconda3\envs\pytorch_env\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[10062.971122074407,
 0.9955029224585735,
 0.9910260686235607,
 0.9955029224585735,
 0.9932594510085306]

## Conventional Model

In [22]:
conventional_weight = torch.load("./conventional_model/conventional.pt", weights_only=True)

In [26]:
conventional_model = SimpleNN(input_size, hidden_size, num_classes)
conventional_model.load_state_dict(conventional_weight)

<All keys matched successfully>

In [27]:
test(conventional_model, testloader=train_loader)

c:\Users\ragha\anaconda3\envs\pytorch_env\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[10069.692924594274,
 0.9955029224585735,
 0.9910260686235607,
 0.9955029224585735,
 0.9932594510085306]